## Analysing the Used CSV

In [2]:
!pwd

/Users/daksh_mac/Desktop/Dev/gitRepos/nitr-hack/AllianceBois/ML


In [33]:
import sys
import os
import pandas as pd

from sklearn.feature_extraction import DictVectorizer

import lightgbm as lgb
import random

In [34]:
filepath="/Users/daksh_mac/Desktop/Dev/gitRepos/nitr-hack/AllianceBois/data/2019-09-27-basel-measures-cleaned-main.csv"

In [35]:
mainFrame=pd.read_csv(filepath,sep=";")

In [36]:
mainFrame.head(10)

,collection,suitcase_id,place_name,place_type,osm_id,cci_id,date,cci,rateCigarrettes,ratePapers,rateBottles,rateExcrements,rateSyringues,rateGums,rateLeaves,rateGrits,rateGlassDebris
0,places,49,Leimgrubenweg,bus_stop,1175332462,287,2019-04-01 8:40:49,3.200000,0.60000,1.2,0.0,0,0,0,0.000000,0,0.0
1,map_features,49,Leimgrubenweg,secondary,25149740,86_9668,2019-04-01 8:40:53,3.357640,0.44236,1.2,0.0,0,0,0,0.000000,0,0.0
2,map_features,49,Reinacherstrasse,primary,117485263,86_11641,2019-04-01 8:40:53,3.357640,0.44236,1.2,0.0,0,0,0,0.000000,0,0.0
3,places,49,Leimgrubenweg,bus_stop,2621172927,287,2019-04-01 8:40:57,3.197570,0.60000,1.2,0.0,0,0,0,0.002430,0,0.0
4,map_features,49,Dornacherstrasse,secondary,148833576,86_12436,2019-04-01 8:42:11,5.000000,0.00000,0.0,0.0,0,0,0,0.000000,0,0.0
5,map_features,49,Reinacherstrasse,primary,148833583,86_12442,2019-04-01 8:42:11,5.000000,0.00000,0.0,0.0,0,0,0,0.000000,0,0.0
6,map_features,49,Reinacherstrasse,primary,148833583,86_12443,2019-04-01 8:42:27,3.800000,0.00000,1.2,0.0,0,0,0,0.000000,0,0.0
7,map_features,49,Münchensteinerstrasse,secondary,332477619,86_14084,2019-04-01 8:45:50,3.557302,0.21984,1.2,0.0,0,0,0,0.022858,0,0.0
8,map_features,49,Nauenstrasse,primary,332478283,86_4134,2019-04-01 8:47:29,3.265280,0.53472,1.2,0.0,0,0,0,0.000000,0,0.0
9,map_features,49,Elisabethenanlage,primary,252383899,86_3678,2019-04-01 8:48:26,3.265280,0.53472,1.2,0.0,0,0,0,0.000000,0,0.0


In [37]:
places_dataframe=pd.read_json(r'/Users/daksh_mac/Desktop/SmartCleanCityHackZurich19-master/cityScraping/cityScraping/spiders/delhiPlaces.json')

In [38]:
place_name_list_foreign=list(mainFrame['place_name'])
place_osmid=list(mainFrame['osm_id'])

In [39]:
replacing_place_names={}
coor_lat={}
coor_long={}
common_place_id={}
corr_osm_lat={}
corr_osm_long={}

for i in range(len(place_name_list_foreign)):
    common_place_id[place_name_list_foreign[i]]=place_osmid[i]

unique_places=list(set(place_name_list_foreign))
num_unique_places=len(unique_places)

'''reading the delhi cities from a json file which is scraped from internet using Scrapy'''


places=list(places_dataframe['place'])
lats=list(places_dataframe['lat'])
longs=list(places_dataframe['long'])

list_of_lats=[]
list_of_longs=[]

for i in range(len(places)):
    coor_lat[places[i]]=lats[i]
    coor_long[places[i]]=longs[i]

random.shuffle(places)

'''Now I have to replace names of the foreign cities with the Indian ones'''

for i in range(num_unique_places):
    replacing_place_names[unique_places[i]]=places[i]
    osmid=common_place_id[unique_places[i]]
    
    corr_osm_lat[osmid]=coor_lat[places[i]]
    corr_osm_long[osmid]=coor_long[places[i]]

for i in range(len(place_name_list_foreign)):
    
    place_name_list_foreign[i]=replacing_place_names[place_name_list_foreign[i]]
    
    list_of_lats.insert(i,coor_lat[place_name_list_foreign[i]])
    list_of_longs.insert(i,coor_long[place_name_list_foreign[i]])
    

In [40]:
mainFrame['place_name']=place_name_list_foreign
mainFrame['latitude']=list_of_lats
mainFrame['longitude']=list_of_longs

In [41]:
mainFrame.head()

,collection,suitcase_id,place_name,place_type,osm_id,cci_id,date,cci,rateCigarrettes,ratePapers,rateBottles,rateExcrements,rateSyringues,rateGums,rateLeaves,rateGrits,rateGlassDebris,latitude,longitude
0,places,49,Firozpur Bāngar,bus_stop,1175332462,287,2019-04-01 8:40:49,3.20000,0.60000,1.2,0.0,0,0,0,0.00000,0,0.0,28.83,76.98
1,map_features,49,Firozpur Bāngar,secondary,25149740,86_9668,2019-04-01 8:40:53,3.35764,0.44236,1.2,0.0,0,0,0,0.00000,0,0.0,28.83,76.98
2,map_features,49,Devnagar,primary,117485263,86_11641,2019-04-01 8:40:53,3.35764,0.44236,1.2,0.0,0,0,0,0.00000,0,0.0,28.65,77.19
3,places,49,Firozpur Bāngar,bus_stop,2621172927,287,2019-04-01 8:40:57,3.19757,0.60000,1.2,0.0,0,0,0,0.00243,0,0.0,28.83,76.98
4,map_features,49,New Delhi,secondary,148833576,86_12436,2019-04-01 8:42:11,5.00000,0.00000,0.0,0.0,0,0,0,0.00000,0,0.0,28.60,77.20


In [42]:
mainFrame.to_csv("/Users/daksh_mac/Desktop/Dev/gitRepos/nitr-hack/AllianceBois/data/important-data.csv",index=False)

## Changing the Basel Collections file

In [43]:
filepath_2="/Users/daksh_mac/Desktop/Dev/gitRepos/nitr-hack/AllianceBois/data/2019-09-27-basel-collections-main.csv"

In [44]:
basel_collections_frame=pd.read_csv(filepath_2)

In [45]:
basel_collections_frame.head(10)

,city_id,osm_id,cci_id,type,geometry,coordinates
0,86,263256727.0,324,Place,Polygon,"POLYGON((7.601737973795679 47.532477, 7.601737..."
1,86,270322890.0,NaN,Place,Polygon,"POLYGON((7.568622384667696 47.561588, 7.568621..."
2,86,274674809.0,NaN,Place,Polygon,"POLYGON((7.589516633078348 47.5480482, 7.58951..."
3,86,583586314.0,251,Place,Polygon,"POLYGON((7.583728694181322 47.5640838, 7.58372..."
4,86,707688638.0,266,Place,Polygon,"POLYGON((7.569987711883181 47.5687268, 7.56998..."
5,86,707688677.0,266,Place,Polygon,"POLYGON((7.573156901658795 47.5660452, 7.57315..."
6,86,707688723.0,266,Place,Polygon,"POLYGON((7.573843098654215 47.5652571, 7.57384..."
7,86,707688727.0,266,Place,Polygon,"POLYGON((7.573092700297358 47.5656881, 7.57309..."
8,86,759733150.0,287,Place,Polygon,"POLYGON((7.564977301710264 47.5660587, 7.56497..."
9,86,759733151.0,287,Place,Polygon,"POLYGON((7.565369203143785 47.5664347, 7.56536..."


In [46]:
len(basel_collections_frame)

2028

In [47]:
main_frame=pd.read_csv("/Users/daksh_mac/Desktop/Dev/gitRepos/nitr-hack/AllianceBois/data/important-data.csv")

In [48]:
main_frame.head()

,collection,suitcase_id,place_name,place_type,osm_id,cci_id,date,cci,rateCigarrettes,ratePapers,rateBottles,rateExcrements,rateSyringues,rateGums,rateLeaves,rateGrits,rateGlassDebris,latitude,longitude
0,places,49,Firozpur Bāngar,bus_stop,1175332462,287,2019-04-01 8:40:49,3.20000,0.60000,1.2,0.0,0,0,0,0.00000,0,0.0,28.83,76.98
1,map_features,49,Firozpur Bāngar,secondary,25149740,86_9668,2019-04-01 8:40:53,3.35764,0.44236,1.2,0.0,0,0,0,0.00000,0,0.0,28.83,76.98
2,map_features,49,Devnagar,primary,117485263,86_11641,2019-04-01 8:40:53,3.35764,0.44236,1.2,0.0,0,0,0,0.00000,0,0.0,28.65,77.19
3,places,49,Firozpur Bāngar,bus_stop,2621172927,287,2019-04-01 8:40:57,3.19757,0.60000,1.2,0.0,0,0,0,0.00243,0,0.0,28.83,76.98
4,map_features,49,New Delhi,secondary,148833576,86_12436,2019-04-01 8:42:11,5.00000,0.00000,0.0,0.0,0,0,0,0.00000,0,0.0,28.60,77.20


In [49]:
len(main_frame)

58242

In [50]:
osms=list(basel_collections_frame['osm_id'])

In [56]:
lats=[]
longs=[]
for i,osmid in enumerate(osms):
     if osmid in corr_osm_lat:
        lats.insert(i,corr_osm_lat[osmid])
     else:
        lats.insert(i,-1)
        
     if osmid in corr_osm_long:
        longs.insert(i,corr_osm_long[osmid])
     else:
        longs.insert(i,-1)

In [57]:
basel_collections_frame["lat"]=lats
basel_collections_frame["long"]=longs

In [59]:
basel_collections_frame.head(40)

,city_id,osm_id,cci_id,type,geometry,coordinates,lat,long
0,86,263256727.0,324,Place,Polygon,"POLYGON((7.601737973795679 47.532477, 7.601737...",-1.00,-1.00
1,86,270322890.0,NaN,Place,Polygon,"POLYGON((7.568622384667696 47.561588, 7.568621...",-1.00,-1.00
2,86,274674809.0,NaN,Place,Polygon,"POLYGON((7.589516633078348 47.5480482, 7.58951...",-1.00,-1.00
3,86,583586314.0,251,Place,Polygon,"POLYGON((7.583728694181322 47.5640838, 7.58372...",-1.00,-1.00
4,86,707688638.0,266,Place,Polygon,"POLYGON((7.569987711883181 47.5687268, 7.56998...",28.65,77.19
5,86,707688677.0,266,Place,Polygon,"POLYGON((7.573156901658795 47.5660452, 7.57315...",-1.00,-1.00
6,86,707688723.0,266,Place,Polygon,"POLYGON((7.573843098654215 47.5652571, 7.57384...",-1.00,-1.00
7,86,707688727.0,266,Place,Polygon,"POLYGON((7.573092700297358 47.5656881, 7.57309...",28.57,77.25
8,86,759733150.0,287,Place,Polygon,"POLYGON((7.564977301710264 47.5660587, 7.56497...",28.53,77.09
9,86,759733151.0,287,Place,Polygon,"POLYGON((7.565369203143785 47.5664347, 7.56536...",-1.00,-1.00


In [62]:
basel_collections_frame.drop(columns=["coordinates"])

,city_id,osm_id,cci_id,type,geometry,lat,long
0,86,263256727.0,324,Place,Polygon,-1.00,-1.00
1,86,270322890.0,NaN,Place,Polygon,-1.00,-1.00
2,86,274674809.0,NaN,Place,Polygon,-1.00,-1.00
3,86,583586314.0,251,Place,Polygon,-1.00,-1.00
4,86,707688638.0,266,Place,Polygon,28.65,77.19
...,...,...,...,...,...,...,...
2023,86,NaN,86_1553940001415e1e99,Feature,LineString,-1.00,-1.00
2024,86,NaN,86_1553940451c07a4335,Feature,LineString,-1.00,-1.00
2025,86,NaN,86_15539405019c060f00,Feature,LineString,-1.00,-1.00
2026,86,NaN,86_15539407381908095d,Feature,LineString,-1.00,-1.00


In [64]:
basel_collections_frame.to_csv("/Users/daksh_mac/Desktop/Dev/gitRepos/nitr-hack/AllianceBois/data/important-data-2.csv",index=False)